In [35]:
#https://www.kaggle.com/datasets/hasibullahaman/traffic-prediction-dataset/data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

In [36]:
file = ('Traffic.csv')
ds = pd.read_csv(file)
ds.head()

,Time,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,TrafficSituation
0,12:00:00 AM,10,Tuesday,31,0,4,4,39,low
1,12:15:00 AM,10,Tuesday,49,0,3,3,55,low
2,12:30:00 AM,10,Tuesday,46,0,3,6,55,low
3,12:45:00 AM,10,Tuesday,51,0,2,5,58,low
4,1:00:00 AM,10,Tuesday,57,6,15,16,94,normal


In [37]:
ds.dtypes

Time                object
Date                 int64
Day of the week     object
CarCount             int64
BikeCount            int64
BusCount             int64
TruckCount           int64
Total                int64
TrafficSituation    object
dtype: object

In [38]:
day_mapping = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
traffic_mapping = {'low': 1, 'normal': 2, 'high': 3 , 'heavy': 4}


In [39]:
ds['Time'] = pd.to_datetime(ds['Time']).dt.hour * 60 + pd.to_datetime(ds['Time']).dt.minute
ds['Time'] = ds['Time'] / 60.0

<ipython-input-39-d87fcb9a3ce3>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ds['Time'] = pd.to_datetime(ds['Time']).dt.hour * 60 + pd.to_datetime(ds['Time']).dt.minute
<ipython-input-39-d87fcb9a3ce3>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ds['Time'] = pd.to_datetime(ds['Time']).dt.hour * 60 + pd.to_datetime(ds['Time']).dt.minute


In [40]:
ds.head()

,Time,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,TrafficSituation
0,0.00,10,Tuesday,31,0,4,4,39,low
1,0.25,10,Tuesday,49,0,3,3,55,low
2,0.50,10,Tuesday,46,0,3,6,55,low
3,0.75,10,Tuesday,51,0,2,5,58,low
4,1.00,10,Tuesday,57,6,15,16,94,normal


In [41]:
ds['Day of the week'].replace(day_mapping, inplace=True)
ds['TrafficSituation'].replace(traffic_mapping, inplace=True)
ds.head()

,Time,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,TrafficSituation
0,0.00,10,2,31,0,4,4,39,1
1,0.25,10,2,49,0,3,3,55,1
2,0.50,10,2,46,0,3,6,55,1
3,0.75,10,2,51,0,2,5,58,1
4,1.00,10,2,57,6,15,16,94,2


In [42]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [51]:
X = ds.drop(columns=['TrafficSituation']) 
y = ds['TrafficSituation'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
result_table = pd.DataFrame(columns=['classifiers', 'fpr','tpr','auc'])
results = []

# YOUR CODE HERE
classifiers = [
    ('Logistic Regression', LogisticRegression(solver='liblinear')),
    ('Naive Bayes', GaussianNB()),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier())
]

for name, classifier in classifiers:
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')  
    recall = recall_score(y_test, y_pred, average='weighted')  
    f1 = f1_score(y_test, y_pred, average='weighted')  
    
    results.append({'classifiers': name, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1_score': f1})   
    
result_table = pd.DataFrame(results)
print(result_table)

           classifiers  accuracy  precision    recall  f1_score
0  Logistic Regression  0.788591   0.770519  0.788591  0.745129
1          Naive Bayes  0.818792   0.856839  0.818792  0.828940
2  K-Nearest Neighbors  0.936242   0.935446  0.936242  0.935614
3        Decision Tree  1.000000   1.000000  1.000000  1.000000
4        Random Forest  0.991611   0.991700  0.991611  0.991625


Random forest is dus de beste zoals je kan zien. Het lijkt alsof decision tree het beste is maar dit is omdat het geoverfit is.
interessant is als je testsize 0.5 maakt krijg je juist een minder hoge precision

In [53]:
from sklearn.model_selection import RandomizedSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300, 400, 500], 
    'max_depth': [None, 10, 20, 30, 40, 50], 
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],  
    'max_features': ['auto', 'sqrt', 'log2'],  
    'bootstrap': [True, False] 
}

rf = RandomForestClassifier()

random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=50, scoring='accuracy', cv=5, n_jobs=-1, random_state=42)

random_search.fit(X_train, y_train)
print("Best Parameters: ", random_search.best_params_)
best_rf_model = random_search.best_estimator_

y_pred = best_rf_model.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print("Test Accuracy: ", accuracy)

C:\Users\adamh\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
115 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
98 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\adamh\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\adamh\anaconda3\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "C:\Users\adamh\anaconda3\lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\adamh\anaconda3\lib\site-package

Best Parameters:  {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}
Test Accuracy:  0.9899328859060402
